In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-wt9giblp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wt9giblp
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936468 sha256=c4e0fd401ea301e73a80b39555b1e1eb6792d688275a3488ff99a93c88f09a23
  Stored in directory: /tmp/pip-ephem-wheel-cache-z1x_5cq1/wheels/c1/95/d1/7d800368ab25942bf88f88fe1716597fcc8d757043c9b48a2f
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transf

In [3]:
!pip install -q -U datasets accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 27.3 MB/s eta 0:00:00


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model

In [6]:
model_name = "google/gemma-3-1b-it"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=HF_TOKEN)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [22]:
dataset = load_dataset('shuttie/dadjokes')

In [23]:
def format_joke(joke):
    return (
        f"<start_of_turn>user\n{joke['question']}<end_of_turn>\n"
        f"<start_of_turn>assistant\n{joke['response']}<end_of_turn>"
    )

formatted_dataset = dataset.map(lambda joke: {'text': format_joke(joke)})

In [25]:
def tokenize_function(batch):
    tokens = tokenizer(
        batch['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

    tokens['labels'] = tokens['input_ids'].copy()

    return tokens

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/52000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [26]:
small_dataset = DatasetDict({
    "train": tokenized_dataset['train'].select(range(10)),
    'test': tokenized_dataset['test'].select(range(10))
})

In [27]:
model = AutoModelForCausalLM.from_pretrained(model_name, attn_implementation='eager', token=HF_TOKEN)

In [28]:
config = LoraConfig(
    r=2,
    lora_alpha=2,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, config)

In [29]:
os.environ["WANDB_DISABLED"] = "true"

In [30]:
training_args = TrainingArguments(
    output_dir='./fine_tuned_gemma',
    eval_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    num_train_epochs=2,
    logging_steps=1,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset['train'],
    eval_dataset=small_dataset['test']
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [32]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss


TypeError: 'HybridCache' object is not iterable